# New event list: Ganglion Cells in the Retina
- **Author:** David Felipe
- **Contact:** https://github.com/davidnfu0
- **Last Modification:** January 25, 2024
- **Description:** In this document, a new list of events will be created by merging two lists of events.
- **Note:** It may not be necessary to do this in all cases. Maybe in your case, it will be necessary to modify the names of the columns.

## Introduction

### Importing libraries

In [1]:
import sys
import pandas as pd

In [2]:
sys.path.append("../../")

In [3]:
from scripts import load_yaml_config

### Paths, constants and configuration

In [4]:
configPath = "../../config/"
config = load_yaml_config(configPath + "general_config.yml")

EVENT_LIST_PATH = "../../" + config["paths"]["data"]["event_list"]
CHIRP_EVENT_LIST_PATH = "../../" + config["paths"]["data"]["chirp_event_list"]

### Loading data

In [5]:
# Loading event list data frames
eventList = pd.read_csv(EVENT_LIST_PATH).drop(44)[
    [
        "n_frames",
        "start_event",
        "end_event",
        "protocol_name",
        "extra_description",
        "repetition_name",
    ]
]
eventList.fillna({"extra_description": ""}, inplace=True)
chirpEventList = pd.read_csv(CHIRP_EVENT_LIST_PATH, sep=";")[
    [
        "n_frames",
        "start_event",
        "end_event",
        "protocol_name",
        "extra_description",
        "rep_name",
    ]
]
chirpEventList.rename(columns={"rep_name": "repetition_name"}, inplace=True)
extraDescription = list(eventList["extra_description"])
naturalImagesIndex = list(
    eventList[eventList["protocol_name"] == "natural_images"].index
)
for i, index in enumerate(naturalImagesIndex):
    extraDescription[index] = "n" + str(i + 1)
eventList["extra_description"] = extraDescription

## Creating the new list of events

In [6]:
chirpSubevents = []
[
    chirpSubevents.append(subevent)
    for subevent in list(chirpEventList["extra_description"].dropna())
    if subevent not in chirpSubevents
]
subeventLenghts = {}

for subevent in chirpSubevents:
    startSubevent = chirpEventList[chirpEventList["extra_description"] == subevent][
        "start_event"
    ].values[0]
    endSubevent = chirpEventList[chirpEventList["extra_description"] == subevent][
        "end_event"
    ].values[0]
    subeventLenghts[subevent] = endSubevent - startSubevent

In [7]:
chirpEvents = eventList[eventList["protocol_name"] == "chirp"]
eventList.drop(chirpEvents.index, inplace=True)
eventListFirst = eventList[eventList.index < 2]
eventListLast = eventList[eventList.index > 2]

In [8]:
newSubevents = {}
CHIRP_REPETITION = list(chirpEvents["repetition_name"])
CHIRP_SUBEVENTS_REPETITION = list(chirpEventList["repetition_name"])
repetitionDifference = []
[
    repetitionDifference.append(item)
    for item in CHIRP_REPETITION
    if item not in CHIRP_SUBEVENTS_REPETITION and item not in repetitionDifference
]
for repetition in repetitionDifference:
    chirp_start = chirpEvents[chirpEvents["repetition_name"] == repetition][
        "start_event"
    ].values[0]
    chirp_end = chirpEvents[chirpEvents["repetition_name"] == repetition][
        "end_event"
    ].values[0]
    lengthCount = 0
    for subevent, length in subeventLenghts.items():
        key = repetition + "_" + subevent
        newSubevents[key] = {}
        newSubevents[key]["n_frames"] = chirpEventList[
            chirpEventList["extra_description"] == subevent
        ]["n_frames"].values[0]
        newSubevents[key]["start_event"] = chirp_start + lengthCount
        newSubevents[key]["end_event"] = (
            chirp_start + lengthCount + length if subevent != "adap_2" else chirp_end
        )
        newSubevents[key]["protocol_name"] = "chirp"
        newSubevents[key]["extra_description"] = subevent
        newSubevents[key]["repetition_name"] = repetition
        lengthCount += length
newSubevents = pd.DataFrame.from_dict(newSubevents, orient="index")
newSubevents.reset_index(inplace=True)
newSubevents.drop("index", axis=1, inplace=True)
newSubevents = pd.concat([chirpEventList, newSubevents], ignore_index=True)
NEW_EVENT_LIST = pd.concat(
    [eventListFirst, newSubevents, eventListLast], ignore_index=True
).reset_index(drop=True)

In [9]:
PROTOCOLS_DESCRIPTION = []
for protocol, description, startTime, endTime, repetition in zip(
    NEW_EVENT_LIST["protocol_name"],
    NEW_EVENT_LIST["extra_description"],
    NEW_EVENT_LIST["start_event"],
    NEW_EVENT_LIST["end_event"],
    NEW_EVENT_LIST["repetition_name"],
):
    key = protocol + "." + description
    if key not in PROTOCOLS_DESCRIPTION:
        PROTOCOLS_DESCRIPTION.append(protocol + "." + description)

## Export the data

In [10]:
NEW_EVENT_LIST.to_csv("../../" + config["paths"]["data"]["new_event_list"], index=False)

___